In [1]:
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense, LSTM
from math import sqrt
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from datetime import datetime

In [2]:
window_size = 48
batch_size_exp = 1
epoch_exp = 7
neurons_exp = 50
predict_values_exp = 6000
lag_exp=72

In [3]:
def timeseries_to_supervised(data, lag=1):
    df = pd.DataFrame(data)
    columns = [df.shift(i) for i in range(1, lag+1)]
    columns.append(df)
    df = pd.concat(columns, axis=1)
    df.fillna(0, inplace=True)
    return df

In [4]:

def difference(dataset, interval=1):
    diff = list()
    for i in range(interval, len(dataset)):
        value = dataset[i] - dataset[i - interval]
        diff.append(value)
    return pd.Series(diff)

In [5]:
def inverse_difference(history, yhat, interval=1):
    return yhat + history[-interval]

In [6]:
def scale(data_norm):
    scaler = MinMaxScaler(feature_range=(-1, 1))
    scaler = scaler.fit(data_norm)
    # transform train
    data_norm = data_norm.reshape(data_norm.shape[0], data_norm.shape[1])
    data_scaled = scaler.transform(data_norm)
    return scaler, data_scaled

In [7]:
def invert_scale(scaler, X, value):
    new_row = [x for x in X] + [value]
    array = np.array(new_row)
    array = array.reshape(1, len(array))
    inverted = scaler.inverse_transform(array)
    return inverted[0, -1]

In [8]:
from keras.layers import Activation, Dense, BatchNormalization, TimeDistributed

def fit_lstm(train, batch_size, nb_epoch, neurons):
    X, y = train[:, 0:-1], train[:, -1]
    X = X.reshape(X.shape[0], 1, X.shape[1])
    model = Sequential()
    model.add(LSTM(neurons_exp, dropout = 0.1, stateful=True))
    model.add(BatchNormalization())
    model.add(Dense(50))
    model.add(Activation('relu'))
    model.add(Dense(50))
    model.add(Activation('tanh'))
    model.add(Dense(1))
    model.compile(loss='mean_squared_error', optimizer='adam')
    for i in range(nb_epoch):
        model.fit(X, y, epochs=1, batch_size=batch_size, verbose=1, shuffle=False)
        
    return model

In [9]:
def forecast_lstm(model, batch_size, X):
    X = X.reshape(1, 1, len(X))
    #print(X)
    yhat = model.predict(X, batch_size=1)
    return yhat[0,0]

In [10]:
import pandas as pd

series = pd.read_csv('../data/wind-data.csv',index_col="Date/Time")
series.head()

,LV ActivePower (kW),Wind Speed (m/s),Theoretical_Power_Curve (KWh),Wind Direction (°)
Date/Time,,,,
01 01 2018 00:00,380.047791,5.311336,416.328908,259.994904
01 01 2018 00:10,453.769196,5.672167,519.917511,268.641113
01 01 2018 00:20,306.376587,5.216037,390.900016,272.564789
01 01 2018 00:30,419.645904,5.659674,516.127569,271.258087
01 01 2018 00:40,380.650696,5.577941,491.702972,265.674286


In [11]:
#del series['Air temperature | (\'C)']
del series['LV ActivePower (kW)']
del series['Wind Direction (°)']
del series['Theoretical_Power_Curve (KWh)']
series.head()

,Wind Speed (m/s)
Date/Time,
01 01 2018 00:00,5.311336
01 01 2018 00:10,5.672167
01 01 2018 00:20,5.216037
01 01 2018 00:30,5.659674
01 01 2018 00:40,5.577941


In [12]:
for i in range(0,10):
  series = series[:-1]
series.tail()

,Wind Speed (m/s)
Date/Time,
31 12 2018 21:30,8.472826
31 12 2018 21:40,8.433422
31 12 2018 21:50,9.976660
31 12 2018 22:00,10.416830
31 12 2018 22:10,10.713670


In [13]:
series.shape

(50520, 1)

In [14]:
raw_values = series.values
diff_values = difference(raw_values, 1)

In [15]:
# transform data to be supervised learning
supervised = timeseries_to_supervised(diff_values, lag_exp)
supervised_values = supervised.values
supervised_values

array([[0, 0, 0, ..., 0, 0, array([0.36083078])],
       [array([0.36083078]), 0, 0, ..., 0, 0, array([-0.45613003])],
       [array([-0.45613003]), array([0.36083078]), 0, ..., 0, 0,
        array([0.44363737])],
       ...,
       [array([-0.03940391]), array([-1.16279221]), array([1.13698769]),
        ..., array([1.36798096]), array([0.23359704]),
        array([1.54323769])],
       [array([1.54323769]), array([-0.03940391]), array([-1.16279221]),
        ..., array([0.41610289]), array([1.36798096]),
        array([0.44017029])],
       [array([0.44017029]), array([1.54323769]), array([-0.03940391]),
        ..., array([-0.62018394]), array([0.41610289]),
        array([0.29683972])]], dtype=object)

In [16]:
# split data into train and test-sets
scaler,supervised_values = scale(supervised_values)
train_scaled, test_scaled = supervised_values[0:-predict_values_exp], supervised_values[-predict_values_exp:]
#print(test_scaled)

In [17]:
len(train_scaled), len(test_scaled)

(44519, 6000)

In [18]:
lstm_model = fit_lstm(train_scaled, batch_size_exp, epoch_exp, neurons_exp)

44519/44519 ━━━━━━━━━━━━━━━━━━━━ 78s 2ms/step - loss: 0.0020
44519/44519 ━━━━━━━━━━━━━━━━━━━━ 73s 2ms/step - loss: 0.0020
44519/44519 ━━━━━━━━━━━━━━━━━━━━ 88s 2ms/step - loss: 0.0020
44519/44519 ━━━━━━━━━━━━━━━━━━━━ 146s 3ms/step - loss: 0.0020
44519/44519 ━━━━━━━━━━━━━━━━━━━━ 112s 3ms/step - loss: 0.0020
44519/44519 ━━━━━━━━━━━━━━━━━━━━ 73s 2ms/step - loss: 0.0020
44519/44519 ━━━━━━━━━━━━━━━━━━━━ 73s 2ms/step - loss: 0.0020


In [ ]:
predictions = list()
expectations = list()
test_pred = list()

for i in range(len(test_scaled)):
    # make one-step forecast
    X, y = test_scaled[i, 0:-1], test_scaled[i, -1]
    yhat = forecast_lstm(lstm_model, 1, X)

    # Replacing value in test scaled with the predicted value.
    test_pred = [yhat] + test_pred
    if len(test_pred) > lag_exp + 1:
        test_pred = test_pred[:-1]
    if i+1<len(test_scaled):
        if i+1 > lag_exp+1:
            test_scaled[i+1] = test_pred
        else:
            test_scaled[i+1] = np.concatenate((test_pred, test_scaled[i+1, i+1:]),axis=0)
    
    # invert scaling
    yhat = invert_scale(scaler, X, yhat)
    # invert differencing
    yhat = inverse_difference(raw_values, yhat, len(test_scaled)+1-i)
    # store forecast
    expected = raw_values[len(train_scaled) + i + 1]
    

    if expected != 0:
        predictions.append(yhat)
        expectations.append(expected)
    print('Hour=%d, Predicted=%f, Expected=%f' % (i+1, yhat, expected))

print(predictions)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 245ms/step
Hour=1, Predicted=3.810286, Expected=5.472752
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step
Hour=2, Predicted=5.037216, Expected=5.492820
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step


C:\Users\HP\AppData\Local\Temp\ipykernel_10908\1020487993.py:31: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  print('Hour=%d, Predicted=%f, Expected=%f' % (i+1, yhat, expected))
C:\Users\HP\AppData\Local\Temp\ipykernel_10908\1020487993.py:31: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  print('Hour=%d, Predicted=%f, Expected=%f' % (i+1, yhat, expected))


Hour=3, Predicted=5.141771, Expected=5.112536
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
Hour=4, Predicted=4.782146, Expected=5.217622
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
Hour=5, Predicted=5.229675, Expected=5.417873
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step


C:\Users\HP\AppData\Local\Temp\ipykernel_10908\1020487993.py:31: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  print('Hour=%d, Predicted=%f, Expected=%f' % (i+1, yhat, expected))
C:\Users\HP\AppData\Local\Temp\ipykernel_10908\1020487993.py:31: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  print('Hour=%d, Predicted=%f, Expected=%f' % (i+1, yhat, expected))
C:\Users\HP\AppData\Local\Temp\ipykernel_10908\1020487993.py:31: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  

Hour=6, Predicted=5.168615, Expected=5.738088
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
Hour=7, Predicted=5.430548, Expected=5.456211
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
Hour=8, Predicted=5.312014, Expected=5.931525


...

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 274ms/step


C:\Users\HP\AppData\Local\Temp\ipykernel_10908\1020487993.py:31: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  print('Hour=%d, Predicted=%f, Expected=%f' % (i+1, yhat, expected))


Hour=4818, Predicted=8.407966, Expected=8.526056
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 169ms/step
Hour=4819, Predicted=8.469618, Expected=8.654116


C:\Users\HP\AppData\Local\Temp\ipykernel_10908\1020487993.py:31: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  print('Hour=%d, Predicted=%f, Expected=%f' % (i+1, yhat, expected))


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 185ms/step


In [ ]:
expectations = np.array(expectations)
predictions = np.array(predictions)
print("Mean Absolute Percent Error: ", (np.mean(np.abs((expectations - predictions) / expectations.mean()))*100))

In [ ]:
# line plot of observed vs predicted
fig = plt.figure(figsize=(20,8))
plt.plot(expectations[500:1000], label="True")
plt.plot(predictions[500:1000], label="Predicted")
plt.legend(loc='upper right')
plt.xlabel("Number of hours")
plt.ylabel("wind speed")
plt.show()

In [ ]:
import pickle

In [ ]:
Pkl_Filename = "../models/wind_speed_model.pkl"  

with open(Pkl_Filename, 'wb') as file:  
    pickle.dump(lstm_model, file)

In [ ]:
with open(Pkl_Filename, 'rb') as file:  
    Pickled_LR1_Model = pickle.load(file)

Pickled_LR1_Model

In [ ]:

score = Pickled_LR1_Model.score(train, test)  
# Print the Score
print("Test score: {0:.2f} %".format(100 * score))  

# Predict the Labels using the reloaded Model
Ypredict = Pickled_LR1_Model.predict(test)  

Ypredict